In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

from elasticsearch import Elasticsearch
import json
import eland as ed
from incremental_learning.config import es_cloud_id, es_user, es_password

In [2]:
es = Elasticsearch(cloud_id=es_cloud_id,
                  http_auth=(es_user, es_password))

In [3]:
key = 'config.sampling_mode'
data_ed = ed.DataFrame(es_client=es, es_index_pattern="experiment-multi-step-sampling", 
             columns=['experiment_uid', key], )
df = data_ed.to_pandas()
df = df.dropna()

In [7]:
for row in df.iterrows():
    uid = row[1].experiment_uid
    value = row[1][key]
    q = {
         "script": {
            "inline": "ctx._source.{key}='{value}'".format(key=key.replace('.', '_'), value=value),
            "lang": "painless"
         },
         "query": {
            "match": {
                "experiment_uid": uid
            }
         }
    }
    es.update_by_query(body=q, index='experiment-multi-step-sampling-metrics')